In [1]:
import torch
import torch_geometric
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F

In [2]:
import pandas as pd
# 数据路径
path = 'D:/data_for_AML/HI-Medium_Trans.csv'

data = pd.read_csv(path)

In [3]:
# 查看data的行数
data.shape

(31898238, 11)

In [10]:
data.dtypes

Timestamp              object
From Bank               int64
Account                object
To Bank                 int64
Account.1              object
Amount Received       float64
Receiving Currency     object
Amount Paid           float64
Payment Currency       object
Payment Format         object
Is Laundering           int64
dtype: object

In [11]:
data.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:17,20,800104D70,20,800104D70,6794.63,US Dollar,6794.63,US Dollar,Reinvestment,0
1,2022/09/01 00:02,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/09/01 00:17,1208,80010E430,1208,80010E430,1880.23,US Dollar,1880.23,US Dollar,Reinvestment,0
3,2022/09/01 00:03,1208,80010E650,20,80010E6F0,73966883.00,US Dollar,73966883.00,US Dollar,Cheque,0
4,2022/09/01 00:02,1208,80010E650,20,80010EA30,45868454.00,US Dollar,45868454.00,US Dollar,Cheque,0


In [13]:
# 对于数据进行预处理：包括

# 对于每一个交易id创建自己的独热编码
def createNodeId(data):
    
    data['NodeID1'] = data['From Bank'].astype(str) + '_' + data['Account'].astype(str)
    data['NodeID2'] = data['To Bank'].astype(str) + '_' + data['Account.1'].astype(str)
    
    del data['Account']
    del data['Account.1']

    return data

# 对于每一个属性特征做feature编码
from sklearn.preprocessing import LabelEncoder
def createFeature(data):
    target = ['Receiving Currency','Payment Currency','Payment Format']
    for col in target:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
    return data

In [14]:
data = createNodeId(data)
data = createFeature(data)

In [15]:
data.head()

,Timestamp,From Bank,To Bank,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,NodeID1,NodeID2
0,2022/09/01 00:17,20,20,6794.63,12,6794.63,12,5,0,20_800104D70,20_800104D70
1,2022/09/01 00:02,3196,3196,7739.29,12,7739.29,12,5,0,3196_800107150,3196_800107150
2,2022/09/01 00:17,1208,1208,1880.23,12,1880.23,12,5,0,1208_80010E430,1208_80010E430
3,2022/09/01 00:03,1208,20,73966883.00,12,73966883.00,12,3,0,1208_80010E650,20_80010E6F0
4,2022/09/01 00:02,1208,20,45868454.00,12,45868454.00,12,3,0,1208_80010E650,20_80010EA30


In [ ]:
# 判断账户是否存在高频交易
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

In [19]:
# 判断账户是否存在高频交易
def isHigherFrequency(data,window = '1H'):
    data['window'] = data['Timestamp'].dt.floor(window)
    data['frequency'] = data.groupby(['NodeID1','window'])['Timestamp'].transform('count')
    return data

In [31]:
data['window']

0          2022-09-01 00:00:00
1          2022-09-01 00:00:00
2          2022-09-01 00:00:00
3          2022-09-01 00:00:00
4          2022-09-01 00:00:00
                   ...        
31898233   2022-09-16 23:00:00
31898234   2022-09-16 23:00:00
31898235   2022-09-16 23:00:00
31898236   2022-09-16 23:00:00
31898237   2022-09-16 23:00:00
Name: window, Length: 31898238, dtype: datetime64[ns]

In [26]:
data['day'] = data['Timestamp'].dt.day

In [20]:
data = isHigherFrequency(data)

C:\Users\41386\AppData\Local\Temp\ipykernel_20604\2902604238.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data['window'] = data['Timestamp'].dt.floor(window)


In [ ]:
# 查看交易次数和洗钱的相关性

corr_xy = data['frequency'].corr(data['Is Laundering'])  # 默认是皮尔逊
print(corr_xy)

0.0021414664962838533


In [24]:
data.head()

,Timestamp,From Bank,To Bank,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,NodeID1,NodeID2,window,frequency
0,2022-09-01 00:17:00,20,20,6794.63,12,6794.63,12,5,0,20_800104D70,20_800104D70,2022-09-01,1
1,2022-09-01 00:02:00,3196,3196,7739.29,12,7739.29,12,5,0,3196_800107150,3196_800107150,2022-09-01,1
2,2022-09-01 00:17:00,1208,1208,1880.23,12,1880.23,12,5,0,1208_80010E430,1208_80010E430,2022-09-01,2
3,2022-09-01 00:03:00,1208,20,73966883.00,12,73966883.00,12,3,0,1208_80010E650,20_80010E6F0,2022-09-01,5
4,2022-09-01 00:02:00,1208,20,45868454.00,12,45868454.00,12,3,0,1208_80010E650,20_80010EA30,2022-09-01,5


In [30]:
# 分布式读取数据，将数据缓存在D盘
# 将交易网络利用PyG构建图数据
# 由于数据量比较大，我们采用分批构建图数据


# 方案1：依照时间节点来分批构建图数据
# 因为TimeStamp是按照天数进行划分，所以我们可以利用时间戳进行分组，当然在分组后我们还是对于每个时间段内的交易进行图构建
# 对于邻接矩阵我们还是采用所有的数据节点做为邻接点

allNodes = pd.unique(data[['NodeID1','NodeID2']].values.ravel())
nodes2Id = {node: i for i, node in enumerate(allNodes)}
numNodes = len(allNodes)

# 由于我们更关系边特征，故此时的节点特征统一为节点所在的账户以及节点在1小时内汇出的交易数额和节点在一小时内汇进的交易数额

featureDim = 3
xGlobal = torch.zeros((numNodes,featureDim),dtype=torch.float)

# 初始化节点特征会按照时间戳变动而变动
# 构建子图


In [48]:
# dataTime : 按照每天每小时进行划分
import torch
from torch_geometric.data import Data, InMemoryDataset, DataLoader
import pandas as pd
def dataToGraph(dataTime,nodes2Id,xGlobal):
    # 边构造
    edgeIndex = torch.tensor([
        [nodes2Id[nid] for nid in dataTime['NodeID1']],  # 起点索引
        [nodes2Id[nid] for nid in dataTime['NodeID2']]   # 终点索引
    ], dtype=torch.long)

    # 边特征：有Amount Received，Amount Pay,Receiving Currency,Payment Currency
    edgeAttr = torch.tensor(dataTime[['Amount Received', 
                                      'Amount Paid', 
                                      'Receiving Currency', 
                                      'Payment Currency']].values, dtype=torch.float).T

    # 边标签
    yLabel = torch.tensor(dataTime['Is Laundering'].values, dtype=torch.float)

    # 节点特征

    # 由于我们在这里的时间片段划分保证了frombank和后续频率是唯一的，故可以直接使用zip，不需要对数据进行去重

    nIdTime = pd.unique(dataTime[['NodeID1','NodeID2']].values.ravel())
    
        # 在这个时间节点内的所有ids
    ids = [nodes2Id[nid] for nid in nIdTime]

        # 在这个时间节点内所有的银行信息
    idsBank = {nodes2Id[nid]:banks for nid,banks in zip(dataTime['NodeID1'],dataTime['From Bank'])}

        # 在这个时间段截断所有汇款频率
    idsFrequency = {nodes2Id[nid]:freq for nid,freq in zip(dataTime['NodeID1'],dataTime['frequency'])}
        # 在这个时间段所有汇入频率
    dataTime['inFrequency'] = dataTime.groupby('NodeID2')['frequency'].transform('sum')
    idsInFrequency = {nodes2Id[nid]:freq for nid,freq in zip(dataTime['NodeID2'],dataTime['inFrequency'])}
        # 更新节点数据
    
    xGlobalTime = xGlobal
    for i in ids:
        xGlobalTime[i][0] = idsBank.get(i,0)  # 如果没有找到对应的银行信息，则赋值为0
        xGlobalTime[i][1] = idsFrequency.get(i,0)
        xGlobalTime[i][2] = idsInFrequency.get(i,0)
    
    # 构建子图
    subGraph = Data(
        x = xGlobalTime,
        edge_index = edgeIndex,
        edge_attr = edgeAttr,
        y = yLabel
    )

    return subGraph

In [39]:
dataTime = data[data['window'] == '2022-09-01 00:00:00']

In [33]:
dataTime.head()

,Timestamp,From Bank,To Bank,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,NodeID1,NodeID2,window,frequency,day
0,2022-09-01 00:17:00,20,20,6794.63,12,6794.63,12,5,0,20_800104D70,20_800104D70,2022-09-01,1,1
1,2022-09-01 00:02:00,3196,3196,7739.29,12,7739.29,12,5,0,3196_800107150,3196_800107150,2022-09-01,1,1
2,2022-09-01 00:17:00,1208,1208,1880.23,12,1880.23,12,5,0,1208_80010E430,1208_80010E430,2022-09-01,2,1
3,2022-09-01 00:03:00,1208,20,73966883.00,12,73966883.00,12,3,0,1208_80010E650,20_80010E6F0,2022-09-01,5,1
4,2022-09-01 00:02:00,1208,20,45868454.00,12,45868454.00,12,3,0,1208_80010E650,20_80010EA30,2022-09-01,5,1


In [49]:
subGraph = dataToGraph(dataTime,nodes2Id,xGlobal)

C:\Users\41386\AppData\Local\Temp\ipykernel_20604\52216659.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataTime['inFrequency'] = dataTime.groupby('NodeID2')['frequency'].transform('sum')


In [53]:
for t in data['window'].unique():
    # 获取当前时间片段的数据
    dataTime = data[data['window'] == t]
    # 构建图
    subGraph = dataToGraph(dataTime,nodes2Id,xGlobal)
    # 保存图数据
    t_str = pd.to_datetime(t).strftime('%Y-%m-%d_%H%M%S')
    torch.save(subGraph, f"D:/dataAMLGraph/subGraph_{t_str}.pt")

C:\Users\41386\AppData\Local\Temp\ipykernel_20604\52216659.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataTime['inFrequency'] = dataTime.groupby('NodeID2')['frequency'].transform('sum')
C:\Users\41386\AppData\Local\Temp\ipykernel_20604\52216659.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataTime['inFrequency'] = dataTime.groupby('NodeID2')['frequency'].transform('sum')
C:\Users\41386\AppData\Local\Temp\ipykernel_20604\52216659.py:36: SettingWithCopyWarning: 
A value is trying to be set o